In [1]:
import os

In [2]:
%pwd

'/Users/vince/code/projects/aws-mlflow-mlops/notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/vince/code/projects/aws-mlflow-mlops'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.aws_mlflow_mlops.constants import *
from src.aws_mlflow_mlops.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    """
    The ConfigurationManager class is responsible for managing the configuration of the data ingestion process.
    It reads configuration, parameters, and schema from YAML files, and creates necessary directories.

    Attributes:
        config (dict): Configuration read from a YAML file.
        params (dict): Parameters read from a YAML file.
        schema (dict): Schema read from a YAML file.
    """

    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        """
        The constructor for ConfigurationManager class.

        Parameters:
            config_filepath (str): The path to the configuration YAML file.
            params_filepath (str): The path to the parameters YAML file.
            schema_filepath (str): The path to the schema YAML file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        The method to get the configuration for the data ingestion process.

        Returns:
            DataIngestionConfig: An object containing the configuration for the data ingestion process.
        """
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile

from src.aws_mlflow_mlops import logger
from src.aws_mlflow_mlops.utils.common import get_size

In [9]:
class DataIngestion:
    """
    The DataIngestion class is responsible for managing the data ingestion process.
    It downloads and extracts data files based on the provided configuration.

    Attributes:
        config (DataIngestionConfig): An object containing the configuration for the data ingestion process.
    """

    def __init__(self, config: DataIngestionConfig):
        """
        The constructor for DataIngestion class.

        Parameters:
            config (DataIngestionConfig): An object containing the configuration for the data ingestion process.
        """
        self.config = config

    def download_file(self):
        """
        The method to download a data file from a URL.
        If the file already exists, it logs the file size.
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL, filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
            )

    def extract_zip_file(self):
        """
        The method to extract a zip file to a specified directory.
        If the directory does not exist, it creates the directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-13 18:25:59,641: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-13 18:25:59,642: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-13 18:25:59,643: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-13 18:25:59,643: INFO: common: created directory at: artifacts]
[2023-11-13 18:25:59,644: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-13 18:26:03,096: INFO: 3602719923: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5D00:DCF0:3A885E6:3D50221:6551F93E
Accept-Ranges: bytes
Date: Mon, 13